# Activity 3.2 – Recoding and Aggregating a Health Care Survey

## Part 1 – Three ways to recode

Before we get the the main event, let’s practice recoding the survey
data in three ways. In all cases, our goal is to assign a score of `5` to
`"Strongly Agree"` down to a score of `1` to `"Strongly Disagree"`.

| Old Label                    | Regular Coded Value |
|------------------------------|---------------------|
| `"Strongly Disagree"`          | 1                   |
| `"Somewhat Disagree"`          | 2                   |
| `"Neither Agree nor Disagree"` | 3                   |
| `"Somewhat Agree"`             | 4                   |
| `"Strongly Agree"`             | 5                   |

**Preparation.** Recall that the `more_dfply.recode` function allows us to
recode a column using a `dict`.

In [5]:
import pandas as pd
from dfply import *
from more_dfply import *
import matplotlib as plt
import seaborn as sns

1.  Load the health survey data found in the data folder.  Inspect the column names, then fix the issues with the `"."`s.  **Hint.** We can use `dfply.rename` with the `dict` unpacking trick.

In [10]:
survey = pd.read_csv('data/health_survey.csv')

In [11]:
new_names = {c.replace(".", "_"):c for c in survey.columns}

In [13]:
survey = (survey
    >> rename(**new_names)
)

In [ ]:
# columns = contains(".").evaluate(survey)
#columns


In [ ]:
# c = "F2.2"

# c.replace(".", "_")

'F2_2'

2.  Use `unique` to verify the labels of various columns. Create a `dict` mapping each survey responses to the corresponding score.

In [ ]:
# survey.F1_1.unique()

array(['Somewhat Agree', 'Strongly Agree', 'Neither Agree nor Disagree',
       'Strongly Disagree', 'Somewhat Disagree'], dtype=object)

In [ ]:
recode = {"Strongly Agree": 5,
          "Somewhat Agree": 4,
          'Neither Agree nor Disagree': 3,
          'Somewhat Disagree': 2,
          'Strongly Disagree': 1}

3.  Test your dictionary with the `map` method on one of the columns.  The columns `F2` and `F1.1` are good test cases.

In [ ]:
survey.F1.map(recode).head(2)

0    4
1    4
Name: F1, dtype: int64

In [ ]:
survey.F2.map(recode).head(2)

0    4.0
1    4.0
Name: F2, dtype: float64

4. Explain why you should be using the `pandas` `Int64` data type here.  Details on the necessity and use of this data type can be found in [Lecture 2.4](https://github.com/wsu-stat489/module2_intro_to_pandas/blob/main/2_4_pandas_dtypes.ipynb)

We should be using Int64 here because it can handle missingness.

#### Method 1 – Brute Force. 

The naïve approach to applying our mapping
is the construct a `mutate`, writing one line per column. This will become
annoying.

1.  Create a pipe that uses `mutate` to transform at least 10 of the columns.

In [ ]:
(survey
    >> mutate(F1_recoded = X.F1.map(recode),
              F1_1_recoded = X.F1_1.map(recode),
              F1_2_recoded = X.F1_2.map(recode),
              F1_3_recoded = X.F1_3.map(recode),
              F1_4_recoded = X.F1_4.map(recode),
              F2_1_recoded = X.F2_1.map(recode),
              F3_1_recoded = X.F3_1.map(recode),
              F4_1_recoded = X.F4_1.map(recode),
              F5_1_recoded = X.F5_1.map(recode),
              F6_1_recoded = X.F6_1.map(recode)
             )
    >> head()
)

,ID,F1,F5,F2,F1_1,F2_1,F6,F4,F3,F5_1,...,F1_recoded,F1_1_recoded,F1_2_recoded,F1_3_recoded,F1_4_recoded,F2_1_recoded,F3_1_recoded,F4_1_recoded,F5_1_recoded,F6_1_recoded
0,1,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,...,4,4,4,4,3,4.0,2.0,4,4,4.0
1,2,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,...,4,4,4,4,4,4.0,3.0,3,3,4.0
2,3,Strongly Agree,Neither Agree nor Disagree,Somewhat Agree,Strongly Agree,Strongly Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,...,5,5,5,5,2,5.0,2.0,3,4,3.0
3,4,Somewhat Agree,Somewhat Agree,Strongly Agree,Somewhat Agree,Strongly Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,Somewhat Disagree,Somewhat Agree,...,4,4,5,4,3,5.0,2.0,3,4,4.0
4,5,Strongly Agree,Strongly Disagree,Neither Agree nor Disagree,Strongly Agree,Somewhat Agree,Strongly Disagree,Strongly Agree,Somewhat Agree,Neither Agree nor Disagree,...,5,5,5,5,4,4.0,4.0,3,3,4.0


2.  Explain,

    1.  in vivid detail, exactly how annoying it would be to continue
        this process.

it would be quite annoying :)

    B.  how this might be prone to buggy code.

while typing out all the columns it would be easy to make name errors

#### Method 2 – dictionary unpacking. 

Recall that we cleaned up repeated,
similar transformations in [Lecture 3.5](https://github.com/wsu-stat489/module3_more_about_mutate/blob/main/3_5_DRY_and_many_transformations.ipynb). Let’s apply that approach here.

1.  Get a `list` of all the questions columns, e.g., using `dfply.columns_from`.

In [ ]:
cols_to_recode = columns_from('F1').evaluate(survey)
cols_to_recode[0:5]

['F1', 'F5', 'F2', 'F1_1', 'F2_1']

2.  Pick one of the columns and write the expression to transform that
    column. Be sure to use the `df["col_string"]` method of referencing
    the column.

In [ ]:
survey['F1'].map(recode).astype("Int64").head()

0    4
1    4
2    5
3    4
4    5
Name: F1, dtype: Int64

3.  Create a variable to hold this column string. Replace the hard coded
    column name with a variable. Rerun to test.  

In [ ]:
c = 'F1'

survey[c].map(recode).astype("Int64").head()

0    4
1    4
2    5
3    4
4    5
Name: F1, dtype: Int64

4.  Convert the single expression to a `dict` using a comprehension
    that iterates over all column names. We want the keys to the be the
    column names and values the resulting recoded columns.  Rerun to
    test. Clean up the code by packaging the complexity in `lambda` functions.

In [ ]:
survey_recode = {c: (survey[c]
                     .map(recode)
                     .astype("Int64")
                    ) 
                 for c in cols_to_recode}

In [ ]:
# survey_recode

5.  Use our dictionary in a mutate using `**` unpacking.  

In [ ]:
(survey
    >> mutate(**survey_recode)
)

,ID,F1,F5,F2,F1_1,F2_1,F6,F4,F3,F5_1,...,F2_9,F3_4,F4_3,F2_10,F1_7,F6_4,F4_4,F5_7,F3_5,F2_11
0,1,4,2,4,4,4,2,4,4,4,...,4,2,3,4,4,4,4,4,4,4
1,2,4,2,4,4,4,2,4,3,3,...,4,4,3,4,4,2,3,4,3,4
2,3,5,3,4,5,5,2,4,4,4,...,4,4,3,4,4,4,4,5,1,4
3,4,4,4,5,4,5,3,3,2,4,...,4,2,4,4,3,3,3,4,2,4
4,5,5,1,3,5,4,1,5,4,3,...,4,4,3,4,4,4,4,5,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,260,4,2,4,3,4,1,4,3,4,...,5,4,3,4,3,3,4,4,4,3
260,261,4,4,5,5,4,2,4,4,4,...,4,4,3,3,3,<NA>,4,4,4,4
261,262,4,3,4,5,5,2,5,4,4,...,4,3,2,4,2,3,5,3,3,4
262,263,5,2,5,5,4,2,4,4,4,...,4,4,2,4,4,2,4,5,2,4


#### Method 3 – Stack Transform Unstack. 

Another method for applying the
same transformation to multiple columns is to (A) stack all the columns
that need transformation, (B) applying the transformation to the stacked
values column, and (C) unstacking the data back into the original shape.
This is the approach you will apply in **Part 2**.

## Part 2 – Performing data preparation

Dr. Bergen, Director of the Statistical Consulting Center at WSU, needs
you to prepare the attached data for analysis. The file
**health_survey.csv** contains the responses to a series of
health-related questions and we need to recode the responses as 1-5
using the definition below. It is important to note that the clients
consider “Strongly Agree” the best answer for most of the questions
coded and want it mapped to 5, but there are a handful of questions for
which they want the mapping reversed as “Strongly Disagree” the
preferred answer. The two types of codings are illustrated in the table
shown below and the list of questions that should receive the reverse
coding are available in the file **ReverseCodingItems.csv**.

| Old Label                    | Regular Coded Value | Reverse Coding |
|------------------------------|---------------------|----------------|
| “Strongly Disagree”          | 1                   | 5              |
| “Somewhat Disagree”          | 2                   | 4              |
| “Neither Agree nor Disagree” | 3                   | 3              |
| “Somewhat Agree”             | 4                   | 2              |
| “Strongly Agree”             | 5                   | 1              |

**Note.** I have prototyped this process in JMP and have provided screenshots of the resulting tables as a guide.

1.  Read in the `ReverseCodingItems.csv` file.  Note that the names in the `"Column Name"` column contain `"."`s.  Fix this, then look at the questions that need reverse coding and explain why it makes sense to reverse the coding on these items.

In [ ]:
reverse_coding = pd.read_csv('data/ReverseCodingItems.csv')

NameError: name 'pd' is not defined

<font color="blue"> *Your thoughts here* </font>

2. Next, you will perform the data preparation by completing each of
    the tasks listed below.  First, *Stack* the response columns.

<img src="img/media/image1.png" style="width:2.91924in;height:1.85212in" alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.30.46%20" />

Work in a pipe and add a temporary `head` at the end.

In [ ]:
# Your code here

3.  Make a new column called *Needs Reverse* by joining on
    the `"Yes"` or `"No"` values from the *Needs Reverse Coding?* from **ReverseCodingItems.csv.**

> <img src="img/media/image2.png"
> style="width:3.78315in;"
> alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.35.26%20" />

Start by copying and adding to the pipe from the previous cell. In practice, you would continue to work in the same cell, but we are illustrating the standard best practice in working with data, by continually cycling through the following steps

1. Add the next step
2. Rerun your code to test.

I want to verify that you are following this practice, which is why we will be copying our previous code to the next step

In [ ]:
# Copy your last pipe and edit the code here

4.   We already created a `dict` for the regular coding.  Make a `dict` for the reverse coding using a `dict` comprehension.  Remember to use the `items` method and two names to iterate through the original `dict`.  Use subtraction.

In [ ]:
# Copy your last pipe and edit the code here

5.   Next we need to make a column with the question type.  For example, all questions that start with `F1`, like `F1` and `F1_1`, need to be coded as `F1`.  Add a `mutate` to create this column using one of the string transformations (extract, split, etc.) from this module.

> <img src="img/media/image6.png"
> style="width:2.97769in;"
> alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.45.03%20" />

In [ ]:
# Copy your last pipe and edit the code here

6.  Make a new column by *Recoding* the Question Types to *F1, F2, …,
    F6.* based on the `Question Type`. **Hint:** You might want to use the `dfply.ifelse`.

> <img src="img/media/image6.png"
> style="width:2.97769in;"
> alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.45.03%20" />

In [ ]:
# Copy your last pipe and edit the code here

7.  *Aggregate* and *unstack.*

> <img src="img/media/image7.png"
> style="width:2.46321in;"
> alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.46.11%20" />
> <img src="img/media/image8.png"
> style="width:2.96893in;"
> alt="../../../../Desktop/Screen%20Shot%202018-03-22%20at%201.46.34%20" />

In [ ]:
# Copy your last pipe and edit the code here

8.  Save the table to a csv file in the `data` folder.

In [ ]:
# Your code here

**Deliverables.** Submit this document with your answer to question 1,
the JMP file containing the results of Part 1, and a csv file with your
final table.